# Luantick Benchmark Example

This example notebook provides a comprehensive example of how to use the Yardstick benchmark framework to collect performance metrics from Luanti game servers and evaluate their performance under bot load.

## Running a Luanti Experiment

The cell below shows you how to run a Luanti server performance experiment using the DAS cluster. This deploys a Luanti server on one node and WalkAround bots on another node.

In [ ]:
from yardstick_benchmark.provisioning import Das
from yardstick_benchmark.monitoring import Telegraf
from yardstick_benchmark.games.luanti.server import LuantiServer
from yardstick_benchmark.games.luanti.workload import RustWalkAround
import yardstick_benchmark
from time import sleep
from datetime import datetime, timedelta
from pathlib import Path
import os
import shutil

# Set up output directory
dest = Path(f"/var/scratch/{os.getlogin()}/yardstick/luanti_output")
if dest.exists():
    shutil.rmtree(dest)

### DEPLOYMENT ENVIRONMENT ###

# The DAS compute cluster is used to provision bare-metal machines
# for our Luanti performance evaluation.
das = Das()
# We reserve 2 nodes - one for the Luanti server, one for bots.
nodes = das.provision(num=2)

try:
    # Clean up any data from previous runs
    yardstick_benchmark.clean(nodes)

    ### METRICS ###

    # Telegraf collects performance metrics from the nodes and applications
    telegraf = Telegraf(nodes)
    # Configure metrics collection for the Luanti server on node 0
    telegraf.add_input_jolokia_agent(nodes[0])
    
    # Deploy and start Telegraf
    res = telegraf.deploy()
    telegraf.start()

    ### LUANTI SERVER ###

    # Deploy Luanti server on node 0
    luanti_server = LuantiServer(nodes[:1], game_mode="minetest_game")
    luanti_server.deploy()
    luanti_server.start()

    ### WORKLOAD - RUST WALKBOTS ###

    # Deploy RustWalkAround bots (from texmodbot) on node 1, connecting to server on node 0
    # These are the Rust-based bots that actually work with the Luanti protocol
    wl = RustWalkAround(
        nodes[1:],              # Bots on node 1
        nodes[0].host,          # Server on node 0
        duration=timedelta(seconds=120),  # 2 minute test
        bots_per_node=15,       # 15 concurrent bots
        movement_mode="random", # Random movement pattern
        movement_speed=2.0,     # Change direction every 2 seconds
    )
    wl.deploy()
    wl.start()

    # Run the benchmark
    sleep_time = 150
    print(f"Running Luanti benchmark for {sleep_time} seconds")
    print(f"Server: {nodes[0].host}, Rust Bots: {nodes[1].host}")
    sleep(sleep_time)

    # Cleanup
    wl.stop()
    wl.cleanup()
    luanti_server.stop()
    luanti_server.cleanup()
    telegraf.stop()
    telegraf.cleanup()

    # Fetch results
    yardstick_benchmark.fetch(dest, nodes)
    print(f"Results saved to: {dest}")
    
finally:
    yardstick_benchmark.clean(nodes)
    das.release(nodes)

## Data Pre-Processing

Process the collected metrics data by splitting the combined CSV files into separate files for each metric type.

In [ ]:
import glob
import pandas as pd

# Find all metrics files from the experiment
raw_data_files = glob.glob(f"{dest}/**/metrics-*.csv", recursive=True)
print(f"Found {len(raw_data_files)} metrics files:")
for f in raw_data_files:
    print(f"  {f}")

# Split combined metrics files into separate files per metric type
for raw_data_file in raw_data_files:
    metrics_file = Path(raw_data_file)
    keys = {}
    with open(metrics_file) as fin:
        for line in fin:
            first_delim = line.find(",")
            second_delim = line.find(",", first_delim+1)
            key = line[first_delim+1:second_delim]
            if key not in keys:
                keys[key] = open(metrics_file.parent / f"{key}.csv", "w+")
            keys[key].write(line)
    for key, fd in keys.items():
        fd.close()
    print(f"Split {metrics_file.name} into {len(keys)} metric files")

## Visualizing Luanti Performance Results

Analyze and visualize the performance data collected during the Luanti benchmark.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load and process CPU utilization data
dfs = []
for cpu_file in glob.glob(f"{dest}/**/cpu.csv", recursive=True):
    df = pd.read_csv(cpu_file, names=["timestamp","measurement","core_id","cpu","host","physical_id","time_active","time_guest","time_guest_nice","time_idle","time_iowait","time_irq","time_nice","time_softirq","time_steal","time_system","time_user"])
    df["node"] = Path(cpu_file).parent.parent.name
    df["timestamp"] = df["timestamp"].transform(lambda x: x - x.min())
    df["timestamp_m"] = df["timestamp"] / 60
    df = df[df.cpu == "cpu-total"]
    df['time_total'] = df.time_active + df.time_idle
    df['util'] = 100 * df.time_active / df.time_total
    df = df.sort_values("util", ascending=False).drop_duplicates(subset=["timestamp", "cpu"], keep="first")
    dfs.append(df)

if dfs:
    df = pd.concat(dfs, ignore_index=True)
    
    # Create CPU utilization plot
    custom_params = {"axes.spines.right": False, "axes.spines.top": False}
    sns.set_theme(style="ticks", rc=custom_params)
    plt.figure(figsize=(12, 6))
    ax = sns.lineplot(df, x="timestamp_m", y="util", hue="node")
    ax.grid(axis="y")
    ax.set_ylim(bottom=0)
    ax.set_ylabel("CPU utilization [%]")
    ax.set_xlabel("Time [minutes]")
    ax.set_title("Luanti Server & Bot CPU Utilization")
    plt.show()
    
    print(f"\nCPU Statistics:")
    print(df.groupby('node')['util'].agg(['mean', 'max', 'std']).round(2))
else:
    print("No CPU data found")

In [ ]:
# Load and visualize memory utilization
mem_files = glob.glob(f"{dest}/**/mem.csv", recursive=True)
if mem_files:
    mem_dfs = []
    for mem_file in mem_files:
        df = pd.read_csv(mem_file)
        df["node"] = Path(mem_file).parent.parent.name
        df["timestamp"] = df["timestamp"].transform(lambda x: x - x.min())
        df["timestamp_m"] = df["timestamp"] / 60
        # Calculate memory utilization percentage
        if 'used_percent' in df.columns:
            mem_dfs.append(df)
    
    if mem_dfs:
        mem_df = pd.concat(mem_dfs, ignore_index=True)
        
        plt.figure(figsize=(12, 6))
        ax = sns.lineplot(mem_df, x="timestamp_m", y="used_percent", hue="node")
        ax.grid(axis="y")
        ax.set_ylim(bottom=0)
        ax.set_ylabel("Memory utilization [%]")
        ax.set_xlabel("Time [minutes]")
        ax.set_title("Luanti Server & Bot Memory Usage")
        plt.show()
        
        print(f"\nMemory Statistics:")
        print(mem_df.groupby('node')['used_percent'].agg(['mean', 'max', 'std']).round(2))
else:
    print("No memory data found")

## Benchmark Summary

This benchmark demonstrates the performance characteristics of Luanti servers under bot load. Key insights:

- **CPU Usage**: Shows how the server handles concurrent bot connections and movement
- **Memory Usage**: Indicates memory consumption patterns for world simulation  
- **Network Traffic**: Reveals protocol overhead and bandwidth requirements

The separation of server and bots onto different nodes allows for clear analysis of where computational bottlenecks occur in the Luanti architecture.